# Cтруктура аудитории и доля
Пример расчета соц-дем структуры аудитории телеканалов.

## Описание задачи и условий расчета
- Период: 01.08.2022 - 31.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+ в разибвке по полу, образованию, занятости, возрастным группам и роду занятий
- Место просмотра: Все места (дом+дача)
- Каналы: ПЕРВЫЙ КАНАЛ, РОССИЯ 1, НТВ									
- Статистики: Share, Rtg000, Rtg%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

In [ ]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["ПЕРВЫЙ КАНАЛ", "РОССИЯ 1", "НТВ"])

# далее в запросах будем использовать id телесетей.

In [ ]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['пол','образование','занятость','возрастная группа (wgh)','род занятий'])

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-08-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (1, 2, 4)'

# Указываем список статистик для расчета
statistics = ['Share', 'Rtg000', 'RtgPer']

# Указываем срезы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем различные срезы по демографии

In [ ]:
# Создадим словарь с демо атрибутами, где ключ - name, а значение entityName атрибута  
demos = {'Пол':'sex', 
             'Образование':'education', 
             'Занятость':'work', 
             'Возрастная группа':'wghSuburbAgeGroup', 
             'Род занятий':'status'}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
results = []
print("Отправляем задания на расчет")

# Каждый элемент списка подставляется в параметр slices
for key, value in demos.items():
    project_name = key
    slices.append(value)

    # Формируем задание для TV API в формате JSON
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Отправляем задание на расчет и ждем выполнения
    task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

    # Получаем результат
    df_timeband = mtask.result2table(mtask.get_result(task_timeband), project_name=project_name)
    
    # Меняем название колонки с демо атрибутом для последующего объединения таблиц
    df_timeband.rename(columns={value: 'valueName'}, inplace=True)
    
    #Сохраняем таблицу
    results.append(df_timeband)
    
    #Удаляем демо атрибут из разбивки
    slices.remove(value)

# Объединяем все полученные таблицы в одну
df = pd.concat(results)
df    

Настраиваем внешний вид таблицы

In [ ]:
# Переименовываем колонку с названием переменной
df.rename(columns={'prj_name':'var_name'}, inplace=True)

df_res = pd.pivot_table(df, values=['Share', 'Rtg000', 'RtgPer'],
                        index=['var_name', 'valueName'], 
                        columns=['tvNetName'])
df_res

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('01_audience_structure')) as writer:
    df_res.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)